In [22]:
from pymongo import MongoClient
import sys
sys.path.append('../statistical_calculations')  # Adds the statistical_calculations directory to the path
import disproportionaly_analysis
from tqdm import tqdm
import json
from datetime import datetime, timedelta 
from calendar import monthrange

# Connect to your MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['vaers']  
col = db['reports']

In [36]:
from datetime import datetime

vaccine = "COVID19"
manufacturer = "JANSSEN"#"PFIZER\\BIONTECH"
symptom = "Facial paralysis"
date = datetime(2021, 2, 28) #yyyy/mm/dd
N_query = {"RECVDATE": {"$lte": date}}
N = col.count_documents(N_query)
print(N)

D_query = {
    "RECVDATE": {"$lte": date},
    "vax_data": {
        "$elemMatch": {
            "VAX_MANU": manufacturer,
            "VAX_TYPE": vaccine
        }
    }
}
D = col.count_documents(D_query)

E_query = {
    "RECVDATE": {"$lte": date},
    "symptoms": {
        "$elemMatch": {
            "$in": [symptom]
        }
    }
}
E = col.count_documents(E_query)

DE_query = {'$and': [D_query, E_query]}
DE = col.count_documents(DE_query)
if DE >= 3: 

    De = D - DE
    dE = E - DE
    de = N - (DE + De + dE)
    
    
    contingency_table = [
        [DE, dE],
        [De, de]
    ]
    print(contingency_table)
    # Compute statistical measures
    rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
    rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
    
    prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
    prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
    
    ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
    ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
    
    chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
    IC = disproportionaly_analysis.information_component(contingency_table)
    IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
    
    # Determine if the reaction meets criteria for a signal
    method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
    method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and ror_ci_lower > 1
        
    if method_1 or method_2:
        print(f"Signal found for {date}")
    else:
        print(f"no Signal found for {date}")
else:
    print(f"No signal found because DE = {DE}")

974447
No signal found because DE = 0


In [12]:
list = [{"manufacturer": "PFIZER\\BIONTECH", "symptom": "Lymphadenopathy", "first_found": "2020-12-24"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Headache", "first_found": "2020-12-16"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Dizziness", "first_found": "2020-12-15"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Facial paralysis", "first_found": "2020-12-18"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Paraesthesia", "first_found": "2020-12-15"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Hypoaesthesia", "first_found": "2020-12-17"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Diarrhoea", "first_found": "2020-12-17"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Nausea", "first_found": "2020-12-16"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Hyperhidrosis", "first_found": "2020-12-16"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Night sweats", "first_found": "2020-12-21"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Arthralgia", "first_found": "2020-12-17"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Myalgia", "first_found": "2020-12-17"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Pain in extremity", "first_found": "2020-12-17"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Heavy menstrual bleeding", "first_found": "2021-01-16"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Fatigue", "first_found": "2020-12-16"}, {"manufacturer": "PFIZER\\BIONTECH", "symptom": "Malaise", "first_found": "2020-12-16"}]

In [13]:
len(list)

16